In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
df_16 = pd.read_csv('hitter_wanted_2016.csv', header=0)
df_17 = pd.read_csv('hitter_wanted_2017.csv', header=0)
df_18 = pd.read_csv('hitter_wanted_2018.csv', header=0)
df_19 = pd.read_csv('hitter_wanted_2019.csv', header=0)
df_20 = pd.read_csv('hitter_wanted_2020.csv', header=0)

In [75]:
df_16.drop(['Unnamed: 0'], axis=1, inplace=True)
df_16['year']=2016
df_17.drop(['Unnamed: 0'], axis=1, inplace=True)
df_17['year']=2017
df_18.drop(['Unnamed: 0'], axis=1, inplace=True)
df_18['year']=2018
df_19.drop(['Unnamed: 0'], axis=1, inplace=True)
df_19['year']=2019
df_20.drop(['Unnamed: 0'], axis=1, inplace=True)
df_20['year']=2020

In [77]:
def preprocessing(data):
    data_count = data.groupby('P_ID').count().reset_index()
    data_count = data_count[['P_ID', 'PA']]
    data = pd.merge(data, data_count, on = ['P_ID'])
    data.rename(columns={'PA_x':'PA', 'PA_y':'count'}, inplace=True)
    data = data.query('count==1')
    return(data)

In [78]:
df_16 = preprocessing(df_16)
df_17 = preprocessing(df_17)
df_17_pa = df_17[['P_ID','PA']]
df_18 = preprocessing(df_18)
df_18_pa = df_18[['P_ID','PA']]
df_19 = preprocessing(df_19)
df_19_pa = df_19[['P_ID','PA']]
df_20 = preprocessing(df_20)
df_20_pa = df_20[['P_ID','PA']]

In [105]:
def new_pa(data1,data2):  
    data1 = data1.drop(['PA'],axis=1)
    key = data2[['P_ID','PA']]
    data1 = pd.merge(data1,key,on='P_ID')
    data1 = data1.query('PA!=0')
    return(data1)

In [106]:
df_16 = new_pa(df_16,df_17)
df_17 = new_pa(df_17,df_18)
df_18 = new_pa(df_18,df_19)
df_19 = new_pa(df_19,df_20)

In [107]:
data1 = pd.concat([df_16,df_17])
data2 = pd.concat([df_18,df_19])
df = pd.concat([data1,data2])

In [108]:
df = df.query('PA>=30')

In [109]:
from sklearn.ensemble import RandomForestRegressor

In [110]:
idx_2019 = np.where(df['year']==2019)
idx_201x = np.where(df['year']!=2019)

x = df.drop(['P_ID','T_ID','count','PA'],axis=1)
y = df['PA']
x_train = x.iloc[idx_201x]
y_train = y.iloc[idx_201x]
x_test = x.iloc[idx_2019]
y_test = y.iloc[idx_2019]

In [111]:
forest = RandomForestRegressor(n_estimators=100)
forest.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [112]:
y_pred = forest.predict(x_test)

from sklearn.metrics import mean_squared_error
mse_test = mean_squared_error(y_test, y_pred)
mse_test

9446.089647972973

In [113]:
y_pred

array([155.55, 133.66,  82.55, 122.4 , 109.87, 128.71, 121.76, 152.87,
        82.48, 147.99, 126.19, 101.63,  89.99, 112.61,  84.33, 115.87,
       126.52, 136.76, 149.53, 149.73,  93.36, 141.63, 129.4 , 100.85,
       107.36,  94.73, 168.24, 138.06, 148.57,  76.92,  90.41, 102.65,
        73.28, 201.05, 123.23, 169.79,  89.22, 164.81, 106.58, 131.71,
        78.29,  91.44,  88.27, 127.3 , 160.76, 111.55, 149.61, 116.4 ,
       119.94, 104.41, 163.59, 127.26,  92.35, 153.96, 145.91, 164.66,
       131.57, 148.17, 119.67, 134.86, 136.45, 155.64,  91.99,  83.4 ,
       133.29, 117.17, 146.36,  86.2 , 159.68, 137.11,  85.95, 146.74,
       130.82, 131.56, 124.72, 109.88, 120.11,  96.72, 136.44, 147.13,
       119.57, 129.47,  79.93, 145.61, 116.93, 177.47, 113.56, 173.06,
       117.27, 130.38, 105.83, 133.19,  95.8 , 189.03, 132.84, 110.42,
       126.05, 151.24, 144.27, 123.62, 165.6 , 137.47, 113.58, 121.8 ,
        81.4 , 191.61,  99.65, 193.88, 157.54, 119.03, 218.69, 146.11,
      

In [114]:
y_test

1      205
2       84
4      134
5      247
6      299
      ... 
175    112
176     92
180     87
181     43
183     31
Name: PA, Length: 148, dtype: int64

In [7]:
df_16['P_ID']

0      60404
1      61102
2      61186
3      62164
4      62700
       ...  
372    66609
373    97336
374    65120
375    97109
376    66469
Name: P_ID, Length: 377, dtype: int64

In [8]:
df_17['P_ID']

0      61700
1      62234
2      62768
3      66209
4      66244
       ...  
347    67893
348    66162
349    60263
350    63903
351    77452
Name: P_ID, Length: 352, dtype: int64